<p align="center">
<img src="https://pyxu-org.github.io/_static/logo.png" alt= "" width=30%>
</p>

# A High Performance Computational Imaging Framework for Python

## The ``Pyxu`` Framework

[Pyxu](https://github.com/pyxu-org/pyxu) is an *open-source computational imaging software framework* for Python 3 with native support for *hardware acceleration* and *distributed computing*. The latter adopts a modular and interoperable *microservice architecture* providing **highly optimised and scalable** general-purpose computational imaging functionalities and tools, easy to **reuse and share across imaging modalities**. 

These include notably:
* a rich collection of matrix-free sensing operators and cost/penalty functionals, which can easily be combined via an **advanced operator algebra logic**,
* a complete base class hierarchy for implementing new, custom operators, 
* a comprehensive algorithmic suite featuring *generic* and *plug-and-play* state-of-the-art proximal methods,
* a *test suite* performing extensive **logical and functional** testing of Pycsou components.   

## Matrix-Free Operators

Operators in Pyxu are **mostly** matrix-free, that is they are defined implicitly via their methods defining forward/backward evaluation, differentiation or proximation. For example, linear operators are defined via the ``apply()`` and ``adjoint()`` methods and **not** via their matrix representation: 

```python
class Sum(LinOp):
    def __init__(self, dim_shape):
        super().__init__(dim_shape=dim_shape, codim_shape=1)
    
    def apply(self, arr):
        return arr.sum()
    
    def adjoint(self, arr):
        return arr * np.ones(self.dim_shape)
```

Matrix-free operators are much more **scalable** (no need to store a huge matrix unecessarily). All methods from the ``LinOp`` base class are matrix-free compatible (e.g., ``lipschitz()``, ``svdvals()``, ``trace()``).

In [ ]:
# Matrix Free operator
import numpy as np
from pyxu.abc.operator import LinOp

class Sum(LinOp):
    def __init__(self, dim_shape):
        super().__init__(dim_shape=dim_shape, codim_shape=1)
    
    def apply(self, arr):
        return arr.sum())
    
    def adjoint(self, arr):
        return arr * np.ones(self.dim_shape)

dim_shape = (10_000, 10_000)
image = np.random.randn(*dim_shape)

sum_matrix_free = Sum(dim_shape=dim_shape)
sum_matrix_form = lambda arr: np.ones(image.size).dot(arr.ravel())

In [ ]:
assert np.isclose(sum_matrix_free(image), sum_matrix_form(image))

In [ ]:
import timeit

In [ ]:
%timeit out1 = sum_matrix_free(image)
%timeit out2 = sum_matrix_form(image)

## Operator Algebra Logic

Complex operators can be constructed by composing Pyxu's fundamental building blocks via the following set of arithmetic operations:

```python
>> op1 + op2 # Addition of two operators
>> op1 * op2 # Composition of two operators
>> op ** 3   # Exponentiation of an operator
>> op.argscale(c), op.argshift(x) # Dilation/shifting
>> 4 * op # Scaling
```
In each case, the type of the output is automatically determined from the set of properties of both operators preserved by the arithmetic operation with all methods inferred from those of the operands. 

In particular, the methods``apply()``, ``jacobian()``, ``grad()``, ``prox()``, ``adjoint()`` are updated via chain rules, allowing users to use the composite operators in proximal gradient algorithms without having to worry about implementing gradients/proximal steps.

In [ ]:
from pyxu.operator import L1Norm, L2Norm

In [ ]:
op1 = L1Norm(dim_shape)
op2 = L2Norm(dim_shape)

(op1 + op2)(image)

In [ ]:
from pyxu.operator import L21Norm, Gradient

In [ ]:
op1 = L21Norm((2,) + dim_shape, l2_axis=0)
op2 = Gradient(dim_shape)
op3 = op1 * op2
print(op3)
op3._expr()

## Lipschitz constants

Pyxu operators all come with (not necessarily tight) Lipschitz/diff-Lipschitz constants which can be accessed via the methods ``lipschitz()`` and ``diff_lipschitz()`` respectively. This is useful for automatically choosing suitable step sizes in optimisation algorithms (done under the hood by Pyxu's algorithmic suite). 


For user-defined operators with unknown Lipschitz constant, calling ``lipschitz()`` for the first time will compute the Frobenius norm of the operator and use it as a rough Lipschitz constant (cached for subsequent calls):

$$ \|A\|_2\leq \|A\|_F=\text{trace}(AA^\ast)=\text{trace}(A^\ast A)\leq \sqrt{\min(N,M)} \|A\|_2$$

This Lipschitz constant is generally not tight (tight for flat spectra), but very cheap to compute (we use the *Hutch++ stochastic algorithm* under the hood). 



In [ ]:
from pyxu.operator import Gradient
dim_shape = (256, 256)

gradient = Gradient(dim_shape)

In [ ]:
%time L_svd = gradient.estimate_lipschitz(method="svd")

In [ ]:
%time L_hutchpp = gradient.estimate_lipschitz(method="trace")

In [ ]:
print(gradient.lipschitz, L_svd, L_hutchpp)